In [8]:
import json
import pandas as pd

file_path = "../analysis_results_DS_r1.json"
# file_path = "../analysis_results_qwq.json"


# 使用 utf-8-sig 編碼重新讀取 JSON 檔案
with open(file_path, "r", encoding="utf-8-sig") as f:
    data = json.load(f)

# 提取每日建議的投資決策
records = []
for entry in data:
    date = entry["date"]
    for stock in entry["stock_recommendations"]:
        if stock["action"] == "買入" and stock["strategy"]["position_size"] > 0:
            records.append({
                "date": date,
                "symbol": stock["symbol"],
                "position_size": stock["strategy"]["position_size"],
                "stop_loss": stock["strategy"]["stop_loss"],
                "take_profit": stock["strategy"]["take_profit"]
            })

# 整理成 DataFrame 供回測使用
df_trades = pd.DataFrame(records)
df_trades["date"] = pd.to_datetime(df_trades["date"].str.replace("⧸", "-"))

# 計算報酬
df_trades["entry_price"] = (df_trades["stop_loss"] + df_trades["take_profit"]) / 2
df_trades["return_rate"] = (df_trades["take_profit"] - df_trades["entry_price"]) / df_trades["entry_price"]
df_trades["capital"] = df_trades["position_size"] * 100000
df_trades["profit"] = df_trades["capital"] * df_trades["return_rate"]

# 輸出報酬總結
total_profit = df_trades["profit"].sum()
total_investment = df_trades["capital"].sum()
total_return_rate = total_profit / total_investment

# 印出結果
print(df_trades[["date", "symbol", "entry_price", "take_profit", "return_rate", "capital", "profit"]])
print("\n--- 總結 ---")
print(f"總投入資金：{total_investment:.2f}")
print(f"總獲利：{total_profit:.2f}")
print(f"總報酬率：{total_return_rate:.2%}")


           date symbol  entry_price  take_profit  return_rate  capital  \
0    2023-01-30   2330   550.520492   602.951015     0.095238  10000.0   
1    2023-01-30   2882    42.680622    46.745443     0.095238  10000.0   
2    2023-01-30     KO    59.358519    65.011712     0.095238  10000.0   
3    2023-01-31   2330   545.441941   597.388792     0.095238  10000.0   
4    2023-01-31   2882    42.432193    46.473354     0.095238  10000.0   
...         ...    ...          ...          ...          ...      ...   
2147 2025-04-10   2882    57.120002    62.560002     0.095238  10000.0   
2148 2025-04-10     KO    73.699503    80.718503     0.095238  10000.0   
2149 2025-04-11   2330   890.400000   975.200000     0.095238  10000.0   
2150 2025-04-11   2882    55.125000    60.375000     0.095238  10000.0   
2151 2025-04-11     KO    74.455504    81.546504     0.095238  10000.0   

          profit  
0     952.380952  
1     952.380952  
2     952.380952  
3     952.380952  
4     952.380952